### Imports

In [426]:
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import numpy as np
import random

### Variables

Estas son las dimensiones del área de simulación, la cantidad de agentes, cajas totales, el número máximo de pasos para la simulación y la cantidad máxima de cajas por celda.

In [427]:
random.seed(67890)
WIDTH, HEIGHT = 20, 20
NUM_AGENTS = 5
TOTAL_BOXES = 200
MAX_STEPS = 100000
MAX_BOXES_PER_CELL = 5

### Clase Cell

Representa una celda en la cuadrícula. Cada celda lleva un conteo de cuántas cajas contiene.

In [428]:
class Cell:
    def __init__(self):
        self.box_count = 0

    @property
    def is_full(self):
        return self.box_count >= MAX_BOXES_PER_CELL

    @property
    def is_empty(self):
        return self.box_count == 0

### Box Organizing Agent
Define el comportamiento de un agente en la simulación. Cada agente puede llevar una caja a la vez (self.carrying_box). Los métodos pick_box y place_box definen cómo recogen y depositan cajas, respectivamente, y move mueve al agente a una celda vecina aleatoria.

In [429]:
class BoxOrganizingAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.carrying_box = False

    def step(self):
        if self.carrying_box:
            self.place_box()
        else:
            self.pick_box()

    def pick_box(self):
        cells = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=True)
        cell_with_boxes = [(cell, self.model.cell_state[cell]) for cell in cells if 0 < self.model.cell_state[cell] < MAX_BOXES_PER_CELL]
        if cell_with_boxes:
            target_cell = min(cell_with_boxes, key=lambda x: x[1])[0]
            self.model.cell_state[target_cell] -= 1
            self.carrying_box = True
            print(f"Agente {self.unique_id} recogió una caja de {target_cell}")
        else:
            print(f"Agente {self.unique_id} no encontró caja para recoger")
        self.move()

    def place_box(self):
        cells = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=True)
        cell_with_space = [(cell, self.model.cell_state[cell]) for cell in cells if self.model.cell_state[cell] < MAX_BOXES_PER_CELL]
        if cell_with_space:
            target_cell = max(cell_with_space, key=lambda x: x[1])[0]
            self.model.cell_state[target_cell] += 1
            self.carrying_box = False
            print(f"Agente {self.unique_id} colocó una caja en {target_cell}")
        else:
            print(f"Agente {self.unique_id} no encontró dónde colocar la caja")
        self.move()

    def move(self):
        possible_moves = [cell for cell in self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)]
        if possible_moves:
            new_position = random.choice(possible_moves)
            self.model.grid.move_agent(self, new_position)
            print(f"Agente {self.unique_id} se movió a {new_position}")


### Box Organizing Model
Es el modelo de la simulación. Contiene una cuadrícula (MultiGrid) donde se colocan los agentes y las celdas, y un planificador (RandomActivation) que activa a los agentes de manera aleatoria. Inicializa el estado de las celdas y coloca las cajas al principio. El método step avanza la simulación un paso, activando cada agente y verificando si se ha alcanzado el objetivo de la simulación.

In [430]:
class BoxOrganizingModel(Model):
    def __init__(self, width, height, num_agents, total_boxes):
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.running = True
        self.cell_state = {(x, y): 0 for x in range(width) for y in range(height)}

        # Inicializar el DataCollector con la función get_grid
        self.datacollector = DataCollector(
            model_reporters={"Grid": self.get_grid, "Agents": self.get_agent_positions}
        )
        # Recopilar datos iniciales
        self.datacollector.collect(self)

        # Crear y colocar agentes
        for i in range(num_agents):
            agent = BoxOrganizingAgent(i, self)
            self.schedule.add(agent)
            x, y = random.randint(0, width - 1), random.randint(0, height - 1)
            self.grid.place_agent(agent, (x, y))

        # Colocar cajas en las celdas
        boxes_placed = 0
        while boxes_placed < total_boxes:
            x, y = random.randint(0, width - 1), random.randint(0, height - 1)
            boxes_to_add = random.randint(1, 3)  # Cantidad aleatoria de cajas entre 1 y 3

            # Verificar que no se exceda el número total de cajas
            if boxes_placed + boxes_to_add > total_boxes:
                boxes_to_add = total_boxes - boxes_placed

            # Asegurarse de no exceder el máximo por celda
            if self.cell_state[(x, y)] + boxes_to_add <= MAX_BOXES_PER_CELL:
                self.cell_state[(x, y)] += boxes_to_add
                boxes_placed += boxes_to_add

    def get_agent_positions(self):
        agent_positions = np.zeros((self.grid.width, self.grid.height))
        for agent in self.schedule.agents:
            x, y = agent.pos
            agent_positions[y][x] = 1  # Marcar la posición del agente
        return agent_positions
    
    def print_step_summary(self):
        print(f"Paso {self.schedule.steps}:")
        for agent in self.schedule.agents:
            print(f"  - Agente {agent.unique_id}: Posición {agent.pos}, Estado {'Con caja' if agent.carrying_box else 'Sin caja'}")


    def get_grid(self):
        grid = np.zeros((self.grid.width, self.grid.height))
        for (x, y), boxes in self.cell_state.items():
            grid[y][x] = boxes
        return grid

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        self.print_step_summary()

        if all(self.cell_state[cell] in [0, MAX_BOXES_PER_CELL] for cell in self.cell_state):
            self.running = False


### Inicialización

Se crea una instancia del modelo y se ejecuta hasta que se alcanza el número máximo de pasos o hasta que todas las celdas contienen 0 o 5 cajas. Al final, imprime el estado final de las celdas y un resumen.

In [431]:
# Definición del modelo y ejecución de la simulación
model = BoxOrganizingModel(WIDTH, HEIGHT, NUM_AGENTS, TOTAL_BOXES)
for _ in range(MAX_STEPS):
    if not model.running:
        break
    model.step()


Agente 2 recogió una caja de (8, 3)
Agente 2 se movió a (7, 3)
Agente 3 no encontró caja para recoger
Agente 3 se movió a (19, 7)
Agente 0 recogió una caja de (7, 4)
Agente 0 se movió a (7, 4)
Agente 1 no encontró caja para recoger
Agente 1 se movió a (13, 15)
Agente 4 recogió una caja de (15, 14)
Agente 4 se movió a (16, 14)
Paso 1:
  - Agente 0: Posición (7, 4), Estado Con caja
  - Agente 1: Posición (13, 15), Estado Sin caja
  - Agente 2: Posición (7, 3), Estado Con caja
  - Agente 3: Posición (19, 7), Estado Sin caja
  - Agente 4: Posición (16, 14), Estado Con caja
Agente 0 colocó una caja en (6, 4)
Agente 0 se movió a (7, 5)
Agente 2 colocó una caja en (6, 3)
Agente 2 se movió a (7, 2)
Agente 3 recogió una caja de (19, 6)
Agente 3 se movió a (19, 6)
Agente 4 colocó una caja en (16, 14)
Agente 4 se movió a (16, 13)
Agente 1 recogió una caja de (14, 15)
Agente 1 se movió a (13, 14)
Paso 2:
  - Agente 0: Posición (7, 5), Estado Sin caja
  - Agente 1: Posición (13, 14), Estado Con caj

In [432]:
# Resumen final
num_cells_with_5 = sum(1 for count in model.cell_state.values() if count == MAX_BOXES_PER_CELL)
num_cells_with_0 = sum(1 for count in model.cell_state.values() if count == 0)
total_steps = model.schedule.steps
print(f"Total de pasos para completar la simulación: {total_steps}")
print(f"Número de celdas con 5 cajas: {num_cells_with_5}")
print(f"Número de celdas con 0 cajas: {num_cells_with_0}")

Total de pasos para completar la simulación: 3322
Número de celdas con 5 cajas: 40
Número de celdas con 0 cajas: 360
